In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

## Data

## Loading data

In [3]:
PATH=Path('data/wikitext-2-raw')

In [4]:
bs = 64
bptt = 70

In [5]:
data = TextLMDataBunch.load(PATH, bs=bs, bptt=bptt)

### AWD-LSTM Model

In [6]:
learn = language_model_learner(data, clip=0.1)

In [14]:
rnn = learn.model.cpu()

In [15]:
rnn

SequentialRNN(
  (0): RNNCore(
    (encoder): Embedding(29836, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(29836, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=29836, bias=True)
    (output_dp): RNNDropout()
  )
)

In [16]:
# encoder weights
rnn_enc = rnn[0].encoder; rnn_enc, rnn_enc.weight.shape

(Embedding(29836, 400, padding_idx=1), torch.Size([29836, 400]))

In [17]:
# decoder weights
rnn_dec = rnn[1].decoder; rnn_dec, rnn_dec.weight.shape

(Linear(in_features=400, out_features=29836, bias=True),
 torch.Size([29836, 400]))

Instead of using decoder, what if we use embeddings as a loss function instead

In [18]:
ob = data.one_batch()

In [19]:
ob[0].shape

torch.Size([64, 73])

Embedding output

In [41]:
emb_out = rnn_enc(ob[0]); emb_out.shape

torch.Size([64, 73, 400])

Encoder output

In [42]:
enc_out = rnn[0](ob[0]);

In [43]:
[x.shape for x in enc_out[0]], [x.shape for x in enc_out[1]]

([torch.Size([64, 73, 1150]),
  torch.Size([64, 73, 1150]),
  torch.Size([64, 73, 400])],
 [torch.Size([64, 73, 1150]),
  torch.Size([64, 73, 1150]),
  torch.Size([64, 73, 400])])

In [44]:
enc_output = enc_out[1][-1]; enc_output.shape

torch.Size([64, 73, 400])

In [100]:
torch.nn.functional.cosine_embedding_loss(emb_out, enc_output, torch.ones(1, device=emb_out.device))

tensor(1.0076, grad_fn=<MeanBackward1>)

In [103]:
(1 - torch.nn.functional.cosine_similarity(emb_out, enc_output, dim=2).mean())

tensor(1.0060, grad_fn=<RsubBackward1>)

In [53]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

In [90]:
torch.nn.functional.cosine_embedding_loss(input1, input2, torch.ones(1))

tensor(1.0143)

In [88]:
torch.nn.functional.cosine_embedding_loss(input1, input1, -torch.ones(100))

tensor(1.)

In [80]:
(1 - torch.nn.functional.cosine_similarity(input1, input2)).sum()

tensor(101.4285)

In [87]:
torch.nn.functional.cosine_similarity(input1, input1, dim=0).shape

torch.Size([128])

In [92]:
(1 - torch.nn.functional.cosine_similarity(input1, input2).mean())

tensor(1.0143)

In [94]:
torch.nn.functional.cosine_similarity(input1, input2, dim=1).mean()

tensor(-0.0143)

In [54]:
output

tensor([-0.0768,  0.0085, -0.0696, -0.0387,  0.0034,  0.0316,  0.0439, -0.0763,
        -0.0297, -0.1067, -0.1284, -0.0343, -0.1463,  0.0633, -0.0067, -0.0830,
         0.0468,  0.0243, -0.0749,  0.1065, -0.0377, -0.0468, -0.0383,  0.0276,
        -0.0681, -0.0206,  0.0089,  0.0316,  0.1816, -0.1098,  0.0750, -0.1207,
        -0.0642, -0.0084,  0.0095, -0.0279, -0.1458, -0.0335, -0.0089, -0.0101,
        -0.1254,  0.1256, -0.0506,  0.0639, -0.0670, -0.0063, -0.0838,  0.0805,
         0.0645, -0.0640, -0.1625,  0.0389,  0.0264,  0.0194,  0.0035,  0.0811,
         0.0585, -0.0669,  0.1008, -0.1518, -0.1017,  0.0467, -0.1301, -0.1225,
        -0.0591,  0.0680,  0.0181, -0.0309, -0.0574, -0.1127,  0.0045, -0.0406,
        -0.0465, -0.0987, -0.0315, -0.0517, -0.0652, -0.0089,  0.0250,  0.0372,
        -0.0272, -0.1885, -0.0644, -0.0051,  0.0326,  0.1659,  0.0613, -0.2073,
        -0.0530,  0.1019,  0.0981, -0.1234,  0.1818,  0.0563,  0.0350,  0.0526,
         0.1102, -0.1092,  0.1745,  0.10